In [ ]:
import requests
import os
import time
from notebook import notebookapp
from pathlib import Path

get_auth = {'authorization': f"AppKey {os.getenv('CELONIS_API_TOKEN')}"}


notebook_id = list(notebookapp.list_running_servers())[0]['base_url'].split('/')[-2]
schedule_id = None
base_url = os.getenv('CELONIS_URL')
cwd = Path.cwd()
home = Path.home()
exec_file = str(Path(str(cwd).replace(str(home), '')[1:]) / 'IBC-Loader-2.0.ipynb') #'testing.ipynb') #

# check if schedule for notebook already exists
url = f'https://{base_url}/machine-learning/api/scheduling'
payload = None
r = requests.get(url, headers=get_auth, json=payload)
jobs = r.json()
for job in jobs:
    if job['name'] == 'Setup_from_Notebook':
        schedule_id = job['id']

# if schedule for notebook doesn't exist, create it
if schedule_id is None:
    url = f'https://{base_url}/machine-learning/api/scheduling'
    payload = {"name": "Setup_from_Notebook",
               "notebookId": notebook_id,
               "executeFile": exec_file,
               "schedulingConfig": {"executionPattern":"DAILY","minute":0,"everyXHours":2,"time":"00:00","day":1,"weekDays":[],"customCron":"","lastExecutionDate":None,"nextExecutionDate":None},
               "timeUnit": "HOURS",
               "timeout": 72,
               "maxRetries": 0,
              }
    r = requests.post(url, headers=get_auth, json=payload)
    schedule_id = r.json()['id']
    url = f'https://{base_url}/machine-learning/api/scheduling/{schedule_id}/change-status?status=false'
    payload = {'status':False}
    r = requests.put(url, headers=get_auth, json=payload)

In [ ]:
# execute the schedule
url = f'https://{base_url}/machine-learning/api/scheduling/{schedule_id}/run'
r = requests.post(url, headers=get_auth)

In [ ]:
# determine execution id
url = f'https://{base_url}/machine-learning/api/scheduling'
r = requests.get(url, headers=get_auth).json()
for i in r:
    if i['id'] == schedule_id:
        exec_id = i['lastNotebookExecutionId']
        break

In [ ]:
# check for completion and status
url = f'https://{base_url}/machine-learning/api/executions/{exec_id}'
while True:
    time.sleep(2)
    r = requests.get(url, headers=get_auth).json()
    if r['completed'] is not None:
        print(r)
        break